In [1]:
import pandas as pd
import numpy as np
import openpyxl

temp = pd.read_excel('data/template_구인구직.xlsx', engine='openpyxl')

In [2]:
# function화
def pre_df(df, temp):
    df.fillna( method='pad', inplace=True)
    df.columns = temp.columns
    df['년/월'] = df['년/월'].str.replace('년 ', '', regex=True).replace('월', '', regex=True)
    df['산업_대분류'] = df['산업_대분류'].str.replace('10차_', '', regex=True)
    df['직종_소분류'] = df['직종_소분류'].str.replace('2018직종_', '', regex=True)
    
    return df


def merge_file(year):
    df = pd.read_excel(f'data/구인구직취업현황(월)_{year}.xlsx', engine='openpyxl')
    temp = pd.read_excel('data/template_구인구직.xlsx', engine='openpyxl')

    import warnings

    warnings.filterwarnings('ignore')

    df = pre_df(df, temp)

    temp['년/월'] = temp['년/월'].astype('str')
    temp['구인인원'].fillna(0, inplace=True)
    temp['구직건수'].fillna(0, inplace=True)
    temp['취업건수'].fillna(0, inplace=True)

    temp['구인인원'] = temp['구인인원'].astype('int64')
    temp['구직건수'] = temp['구직건수'].astype('int64')
    temp['취업건수'] = temp['취업건수'].astype('int64')

    temp_year = temp.loc[temp['년/월'].str.startswith(f'{year}')]

    merged = pd.merge(temp_year, df, how='left', on=['년/월', '시도', '산업_대분류', '직종_소분류'])

    merged.drop(columns=['구인인원_x', '구직건수_x', '취업건수_x'], inplace=True)

    merged.columns = temp.columns

    merged.fillna(0, inplace=True)

    # merged.to_csv(f'data/구인구직취업현황_{year}_merged.csv', index=False)
    
    return merged

In [3]:
df_2018 = merge_file(2018)
df_2019 = merge_file(2019)
df_2020 = merge_file(2020)
df_2021 = merge_file(2021)
df_2022 = merge_file(2022)

In [4]:
dfs = pd.concat([df_2018, df_2019, df_2020, df_2021, df_2022], axis=0, ignore_index=True)
dfs.to_csv('data/구인구직취업현황_merged.csv', index=False)

In [5]:
print(len(temp), len(dfs))

140760 140760


In [6]:
dfs

,년/월,시도,산업_대분류,직종_소분류,구인인원,구직건수,취업건수
0,201801,서울,"농업, 임업 및 어업",컴퓨터하드웨어·통신공학 기술자,0.0,0.0,0.0
1,201801,서울,"농업, 임업 및 어업",컴퓨터시스템 전문가,0.0,0.0,0.0
2,201801,서울,"농업, 임업 및 어업",소프트웨어 개발자,0.0,0.0,0.0
3,201801,서울,"농업, 임업 및 어업",데이터·네트워크 및 시스템 운영 전문가,0.0,0.0,0.0
4,201801,서울,"농업, 임업 및 어업",정보보안 전문가,0.0,0.0,0.0
...,...,...,...,...,...,...,...
140755,202212,제주,해당없음,컴퓨터시스템 전문가,0.0,2.0,0.0
140756,202212,제주,해당없음,소프트웨어 개발자,0.0,36.0,0.0
140757,202212,제주,해당없음,데이터·네트워크 및 시스템 운영 전문가,0.0,13.0,0.0
140758,202212,제주,해당없음,정보보안 전문가,0.0,1.0,0.0


In [8]:
dfs_merged = dfs.drop(columns=['직종_소분류']).groupby(['년/월', '시도', '산업_대분류'], sort=False).sum().reset_index()
dfs_merged

,년/월,시도,산업_대분류,구인인원,구직건수,취업건수
0,201801,서울,"농업, 임업 및 어업",0.0,0.0,0.0
1,201801,서울,광업,0.0,0.0,0.0
2,201801,서울,제조업,33.0,0.0,1.0
3,201801,서울,"전기, 가스, 증기 및 공기조절 공급업",1.0,0.0,0.0
4,201801,서울,"수도, 하수 및 폐기물 처리, 원료 재생업",0.0,0.0,0.0
...,...,...,...,...,...,...
23455,202212,제주,"협회 및 단체, 수리 및 기타 개인 서비스업",0.0,0.0,0.0
23456,202212,제주,가구 내 고용활동 및 달리 분류되지 않은 자가소비 생산활동,0.0,0.0,0.0
23457,202212,제주,국제 및 외국기관,0.0,0.0,0.0
23458,202212,제주,분류불능,0.0,0.0,16.0


In [9]:
dfs_merged.to_csv('data/구인구직취업현황_merged.csv', index=False)